In [1]:
import os, ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
    getattr(ssl, '_create_unverified_context', None)): 
    ssl._create_default_https_context = ssl._create_unverified_context

import tensorflow as tf
import numpy as np

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)

#X_train = X_train - (np.mean(X_train, 0)+10**-8)[np.newaxis, :]
#X_test = X_test - (np.mean(X_train, 0)+10**-8)[np.newaxis, :]

X_train_full, y_train_full = X_train, y_train

X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]



c:\users\ferbi\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\ferbi\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\ferbi\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\ferbi\appd

In [2]:
import tensorflow as tf
import numpy as np
from datetime import datetime
import math

def execute_code(rand_seed = 1, lambda_exp = -6.0):
    LOC = """

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

print("Now:", now)

n_inputs = X_train.shape[1]
n_hidden1 = 1024
n_hidden2 = 1024
n_hidden3 = 1024
n_hidden4 = 1024
n_hidden5 = 1024
n_outputs = 10

def reset_graph(seed=43):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph(@rand_seed@)

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

epsVal = pow(10.0, -8.0)

def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2/np.sqrt(n_inputs + n_neurons)
        init = tf.random_normal((n_inputs, n_neurons), mean=0.0, stddev=stddev)
        
        W = tf.Variable(init, name="kernel")
        #W = tf.cast(tf.abs(W) >= epsVal, W.dtype) * W
        b = tf.Variable(tf.zeros([n_neurons]), name="bias")
        Z = tf.add(tf.matmul(X, W), b)
                
        if activation is not None:
            return activation(Z)
        else:
            return Z
        
with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, name="hidden1", activation=tf.nn.relu)
    hidden2 = neuron_layer(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
    hidden3 = neuron_layer(hidden2, n_hidden3, name="hidden3", activation=tf.nn.relu)
    hidden4 = neuron_layer(hidden3, n_hidden4, name="hidden4", activation=tf.nn.relu)
    hidden5 = neuron_layer(hidden4, n_hidden5, name="hidden5", activation=tf.nn.relu)
    logits = neuron_layer(hidden5, n_outputs, name="outputs")

hidden1_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden1/kernel:0")
hidden2_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden2/kernel:0")
hidden3_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden3/kernel:0")
hidden4_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden4/kernel:0")
hidden5_W = tf.get_default_graph().get_tensor_by_name("dnn/hidden5/kernel:0")
logits_W = tf.get_default_graph().get_tensor_by_name("dnn/outputs/kernel:0")
    
def calcRegValue(W, type=1, eps=0.00001):
    if type == 1:
        wMax = 1.0
        gt_eps = tf.minimum(1.0, tf.add(1.0, tf.constant(eps/(wMax-eps))*tf.subtract(tf.abs(W), wMax)))
        #gt_eps = 1.0  
        #lt_eps = tf.constant((1-eps)/eps)*tf.abs(W)
        lt_eps = 0.0
        reg_value = tf.cast(tf.abs(W) >= eps, W.dtype)*gt_eps + tf.cast(tf.abs(W) < eps, W.dtype)*lt_eps
        return tf.reduce_sum(reg_value)
    elif type == 2:
        reg_value = tf.cast(tf.abs(W) >= eps, W.dtype)*W
        return tf.reduce_sum(reg_value)
    elif type == 3:
        epsMat = tf.constant(eps, shape=W.get_shape())
        return tf.reduce_sum(tf.subtract(1.0, tf.pow(epsMat, tf.pow(W, 2))))
    elif type == 4:
        epsMat = tf.constant(eps, shape=W.get_shape())
        return tf.reduce_sum(tf.subtract(1.0, tf.pow(epsMat, tf.abs(W))))
    elif type == 5:
        return tf.reduce_sum(tf.abs(W))
    elif type == 6:
        return tf.reduce_sum(tf.pow(W, 2))
    elif type == 7:
        gt_eps = 1.0  
        epsMat = tf.constant(eps, shape=W.get_shape())
        lt_eps = 1.0 - tf.pow(epsMat, tf.abs(W))
        reg_value = tf.cast(tf.abs(W) >= np.sqrt(eps), W.dtype)*gt_eps + tf.cast(tf.abs(W) < np.sqrt(eps), W.dtype)*lt_eps
        return tf.reduce_sum(reg_value)
    elif type == 8:
        eMat = tf.constant(math.e, shape=W.get_shape())
        return tf.reduce_sum(tf.subtract(1.0, tf.pow(eMat, tf.subtract(0.0, 750*tf.pow(W, 2)))))
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    regType = 8
    total_regvalue = tf.add_n([calcRegValue(hidden1_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden2_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden3_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden4_W, type=regType, eps=epsVal), \
                               calcRegValue(hidden5_W, type=regType, eps=epsVal), \
                               calcRegValue(logits_W, type=regType, eps=epsVal)])
    
    total_nzw = tf.cast(tf.add_n([tf.count_nonzero(hidden1_W), \
                                  tf.count_nonzero(hidden2_W), \
                                  tf.count_nonzero(hidden3_W), \
                                  tf.count_nonzero(hidden4_W), \
                                  tf.count_nonzero(hidden5_W), \
                                  tf.count_nonzero(logits_W)]), tf.float32)
    total_nzw_eps = tf.cast(tf.add_n([tf.reduce_sum(tf.cast(abs(hidden1_W) < epsVal, hidden1_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden2_W) < epsVal, hidden2_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden3_W) < epsVal, hidden3_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden4_W) < epsVal, hidden4_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(hidden5_W) < epsVal, hidden5_W.dtype)), \
                                      tf.reduce_sum(tf.cast(abs(logits_W) < epsVal, logits_W.dtype))]), tf.int64)
    
    lambdaParam = tf.pow(10.0, @lambda_exp@, name="lambdaParam")
    loss_total = tf.add(tf.reduce_mean(xentropy, name="loss_total"), tf.scalar_mul(lambdaParam, total_regvalue))
    #loss_total = tf.reduce_mean(xentropy, name="loss_total")
    
learning_rate = tf.placeholder(tf.float32, shape=(), name="learning_rate")
with tf.name_scope("train"):
    #optimizer = tf.train.ProximalGradientDescentOptimizer(learning_rate)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    #optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
    #optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
    training_op = optimizer.minimize(loss_total)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()
saver = tf.train.Saver()

reg_summary = tf.summary.scalar('Count_Reg', total_regvalue)
loss_summary = tf.summary.scalar('Loss_Total', loss_total)
err_summary = tf.summary.scalar('Class_Err_Perc', (1-accuracy)*100)
#file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

n_epochs = 250
batch_size = 20

def shuffle_batch(X, y, batch_size):
    np.random.seed(43)
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    batch_num = -1
    for batch_idx in np.array_split(rnd_idx, n_batches):
        batch_num = batch_num + 1
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield batch_num, X_batch, y_batch

def get_lr_step(epochCounter, start_lr, stepsize, end_lr, lr_iter_period):
    lr = start_lr - math.floor(epochCounter/lr_iter_period) * stepsize
    return max(lr, end_lr)
        
def get_lr_triangle(epochCounter, lr_min, lr_max, lr_iter_period):
    cycle = math.floor(1 + epochCounter/(2*lr_iter_period))
    x = abs(epochCounter/lr_iter_period - 2*cycle + 1)
    lr = lr_min + (lr_max - lr_min) * max(0, (1-x)/math.pow(2, cycle-1))
    return lr

max_checks_without_progress = 20
checks_without_progress = 0
best_loss = np.infty

start_time = datetime.now()

with tf.Session() as sess:
    init.run()

    for epoch in range(n_epochs):
        lr = get_lr_triangle(epoch, lr_min=0.01, lr_max=0.25, lr_iter_period=20)
        #lr=0.01
        rnd_idx = np.random.permutation(len(X_train))
        for rnd_indices in np.array_split(rnd_idx, len(X_train) // batch_size):
            X_batch, y_batch = X_train[rnd_indices], y_train[rnd_indices]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch, learning_rate: lr})
        
        loss_val, reg_loss_val, lambda1, acc_val = sess.run([loss_total, total_regvalue, lambdaParam, accuracy], \
                                                   feed_dict={X: X_valid, y: y_valid, learning_rate: lr})
        base_loss_val = loss_val - lambda1*reg_loss_val
        
        if best_loss > loss_val:
            save_path = saver.save(sess, "./models/model_" + now + ".ckpt")
            best_loss = loss_val
            checks_without_progress = 0
        else:
            checks_without_progress += 1
            if checks_without_progress > max_checks_without_progress:
                print("Early stopping!")
                break
        print("{}\tValidation losses: {:.4f}, {:.4f}, {:.4f}\tBest Loss: {:.4f} ({})\tAccuracy: {:.2f}%".format(
            epoch, base_loss_val, lambda1*reg_loss_val, loss_val, best_loss, checks_without_progress, acc_val * 100))

end_time = datetime.now()
running_time = end_time - start_time
print("Total running time:\t", running_time.seconds)

with tf.Session() as sess:
    saver.restore(sess, "./models/model_" + now + ".ckpt") # or better, use save_path
    #saver.restore(sess, "./models/model_20190129210937.ckpt") # or better, use save_path
    n_batches = len(X_test) // 2000
    batch_remaining = n_batches
    correct_count = 0.0
    for batch_idx in np.array_split(range(len(X_test)), n_batches):
        correct_count = correct_count + len(batch_idx)*accuracy.eval(feed_dict={X: X_test[batch_idx], y: y_test[batch_idx]})
        print("Batches remaining:    ", batch_remaining)
        batch_remaining = batch_remaining - 1
print("Test error rate:    {:.10f}%".format(100*(1-correct_count/len(X_test)))) 
"""
    LOC = LOC.replace("@rand_seed@", str(rand_seed))
    LOC = LOC.replace("@lambda_exp@", str(lambda_exp))
    exec(LOC)

In [ ]:
for j in [-4.0, -4.2, -4.4, -4.6, -4.8, -5.0, -5.2, -5.4, -5.6, -5.8, -6.0, -6.2, -6.4, -6.6, -6.8, -7.0]:
    for i in [1, 2, 3]:
        print("RANDOM SEED:  ", i, "\tLAMBDA_EXP:  ", j)
        execute_code(rand_seed=i, lambda_exp=j)

RANDOM SEED:   1 	LAMBDA_EXP:   -4.0
Now: 20240513145448
0	Validation losses: 0.5705, 82.7464, 83.3168	Best Loss: 83.3168 (0)	Accuracy: 77.48%
1	Validation losses: 0.4639, 42.1234, 42.5873	Best Loss: 42.5873 (0)	Accuracy: 84.44%
2	Validation losses: 0.4568, 29.1944, 29.6513	Best Loss: 29.6513 (0)	Accuracy: 83.64%
3	Validation losses: 0.4792, 22.9532, 23.4323	Best Loss: 23.4323 (0)	Accuracy: 82.26%
4	Validation losses: 0.4621, 19.3098, 19.7720	Best Loss: 19.7720 (0)	Accuracy: 82.96%
5	Validation losses: 0.5119, 16.9505, 17.4624	Best Loss: 17.4624 (0)	Accuracy: 80.92%
6	Validation losses: 0.4025, 15.2771, 15.6796	Best Loss: 15.6796 (0)	Accuracy: 85.46%
7	Validation losses: 0.4420, 14.0269, 14.4690	Best Loss: 14.4690 (0)	Accuracy: 83.62%
8	Validation losses: 0.4133, 13.0705, 13.4838	Best Loss: 13.4838 (0)	Accuracy: 85.08%
9	Validation losses: 0.6003, 12.3138, 12.9140	Best Loss: 12.9140 (0)	Accuracy: 79.56%
10	Validation losses: 0.3995, 11.6842, 12.0837	Best Loss: 12.0837 (0)	Accuracy: 85.

97	Validation losses: 0.3273, 7.6721, 7.9994	Best Loss: 7.9880 (2)	Accuracy: 88.16%
98	Validation losses: 0.3251, 7.6446, 7.9697	Best Loss: 7.9697 (0)	Accuracy: 88.76%
99	Validation losses: 0.3180, 7.6229, 7.9409	Best Loss: 7.9409 (0)	Accuracy: 88.88%
100	Validation losses: 0.3372, 7.6129, 7.9501	Best Loss: 7.9409 (1)	Accuracy: 87.98%
101	Validation losses: 0.3416, 7.6354, 7.9770	Best Loss: 7.9409 (2)	Accuracy: 87.72%
102	Validation losses: 0.3309, 7.6017, 7.9326	Best Loss: 7.9326 (0)	Accuracy: 88.42%
103	Validation losses: 0.3379, 7.5775, 7.9155	Best Loss: 7.9155 (0)	Accuracy: 88.66%
104	Validation losses: 0.3401, 7.5831, 7.9231	Best Loss: 7.9155 (1)	Accuracy: 88.50%
105	Validation losses: 0.3205, 7.5533, 7.8738	Best Loss: 7.8738 (0)	Accuracy: 88.84%
106	Validation losses: 0.3347, 7.5357, 7.8704	Best Loss: 7.8704 (0)	Accuracy: 88.52%
107	Validation losses: 0.3164, 7.5241, 7.8405	Best Loss: 7.8405 (0)	Accuracy: 89.32%
108	Validation losses: 0.3297, 7.5113, 7.8410	Best Loss: 7.8405 (1)	

194	Validation losses: 0.3645, 7.1534, 7.5179	Best Loss: 7.5179 (0)	Accuracy: 89.26%
195	Validation losses: 0.3740, 7.1527, 7.5267	Best Loss: 7.5179 (1)	Accuracy: 89.18%
196	Validation losses: 0.3658, 7.1434, 7.5092	Best Loss: 7.5092 (0)	Accuracy: 89.28%
197	Validation losses: 0.3662, 7.1427, 7.5090	Best Loss: 7.5090 (0)	Accuracy: 89.16%
198	Validation losses: 0.3717, 7.1386, 7.5104	Best Loss: 7.5090 (1)	Accuracy: 89.42%
199	Validation losses: 0.3686, 7.1339, 7.5026	Best Loss: 7.5026 (0)	Accuracy: 89.08%
200	Validation losses: 0.3726, 7.1328, 7.5055	Best Loss: 7.5026 (1)	Accuracy: 89.14%
201	Validation losses: 0.3811, 7.1325, 7.5136	Best Loss: 7.5026 (2)	Accuracy: 89.32%
202	Validation losses: 0.3714, 7.1315, 7.5029	Best Loss: 7.5026 (3)	Accuracy: 89.46%
203	Validation losses: 0.3661, 7.1301, 7.4962	Best Loss: 7.4962 (0)	Accuracy: 89.28%
204	Validation losses: 0.3688, 7.1302, 7.4989	Best Loss: 7.4962 (1)	Accuracy: 89.02%
205	Validation losses: 0.3753, 7.1304, 7.5057	Best Loss: 7.4962 (

39	Validation losses: nan, nan, nan	Best Loss: 9.3297 (20)	Accuracy: 0.00%
Early stopping!
Total running time:	 3230
INFO:tensorflow:Restoring parameters from ./models/model_20240513203034.ckpt
Batches remaining:     5
Batches remaining:     4
Batches remaining:     3
Batches remaining:     2
Batches remaining:     1
Test error rate:    16.5599989891%
RANDOM SEED:   3 	LAMBDA_EXP:   -4.0
Now: 20240513212425
0	Validation losses: 0.5578, 82.6280, 83.1858	Best Loss: 83.1858 (0)	Accuracy: 82.26%
1	Validation losses: 0.4837, 42.0425, 42.5263	Best Loss: 42.5263 (0)	Accuracy: 83.00%
2	Validation losses: 0.4397, 29.1773, 29.6171	Best Loss: 29.6171 (0)	Accuracy: 84.52%
3	Validation losses: 0.4958, 22.9378, 23.4337	Best Loss: 23.4337 (0)	Accuracy: 81.66%
4	Validation losses: 0.4508, 19.3108, 19.7615	Best Loss: 19.7615 (0)	Accuracy: 83.30%
5	Validation losses: 0.4757, 16.9235, 17.3992	Best Loss: 17.3992 (0)	Accuracy: 82.12%
6	Validation losses: 0.4495, 15.2327, 15.6822	Best Loss: 15.6822 (0)	Accu

11	Validation losses: 0.5174, 8.2540, 8.7714	Best Loss: 8.7714 (0)	Accuracy: 81.42%
12	Validation losses: 0.4304, 7.9168, 8.3473	Best Loss: 8.3473 (0)	Accuracy: 84.30%
13	Validation losses: 0.4433, 7.6431, 8.0864	Best Loss: 8.0864 (0)	Accuracy: 84.20%
14	Validation losses: 0.4697, 7.3784, 7.8481	Best Loss: 7.8481 (0)	Accuracy: 82.42%
15	Validation losses: 0.3839, 7.1755, 7.5595	Best Loss: 7.5595 (0)	Accuracy: 86.64%
16	Validation losses: 0.4097, 6.9793, 7.3890	Best Loss: 7.3890 (0)	Accuracy: 85.00%
17	Validation losses: 0.4683, 6.8819, 7.3503	Best Loss: 7.3503 (0)	Accuracy: 82.94%
18	Validation losses: 0.4184, 6.7385, 7.1568	Best Loss: 7.1568 (0)	Accuracy: 85.74%
19	Validation losses: 0.4056, 6.7390, 7.1446	Best Loss: 7.1446 (0)	Accuracy: 86.02%
20	Validation losses: 0.3621, 6.4856, 6.8477	Best Loss: 6.8477 (0)	Accuracy: 86.76%
21	Validation losses: 0.3542, 6.3090, 6.6633	Best Loss: 6.6633 (0)	Accuracy: 86.96%
22	Validation losses: 0.4789, 6.2249, 6.7039	Best Loss: 6.6633 (1)	Accuracy:

...

Batches remaining:     5
Batches remaining:     4
Batches remaining:     3
Batches remaining:     2
Batches remaining:     1
Test error rate:    10.7300007343%
RANDOM SEED:   1 	LAMBDA_EXP:   -5.0
Now: 20240514204029
0	Validation losses: 0.4117, 21.8114, 22.2231	Best Loss: 22.2231 (0)	Accuracy: 85.12%
1	Validation losses: 0.3619, 16.0273, 16.3892	Best Loss: 16.3892 (0)	Accuracy: 87.34%
2	Validation losses: 0.3824, 10.8595, 11.2418	Best Loss: 11.2418 (0)	Accuracy: 86.06%
3	Validation losses: 0.4016, 7.8175, 8.2192	Best Loss: 8.2192 (0)	Accuracy: 85.46%
4	Validation losses: 0.3710, 6.1116, 6.4826	Best Loss: 6.4826 (0)	Accuracy: 86.22%
5	Validation losses: 0.4556, 5.0558, 5.5114	Best Loss: 5.5114 (0)	Accuracy: 83.72%
6	Validation losses: 0.3356, 4.3389, 4.6745	Best Loss: 4.6745 (0)	Accuracy: 87.50%
7	Validation losses: 0.4269, 3.8277, 4.2547	Best Loss: 4.2547 (0)	Accuracy: 84.42%
8	Validation losses: 0.3561, 3.4424, 3.7985	Best Loss: 3.7985 (0)	Accuracy: 86.92%
9	Validation losses: 0.4528

17	Validation losses: 0.3436, 2.0937, 2.4374	Best Loss: 2.4374 (0)	Accuracy: 87.84%
18	Validation losses: nan, nan, nan	Best Loss: 2.4374 (1)	Accuracy: 0.00%
19	Validation losses: nan, nan, nan	Best Loss: 2.4374 (2)	Accuracy: 0.00%
20	Validation losses: nan, nan, nan	Best Loss: 2.4374 (3)	Accuracy: 0.00%
21	Validation losses: nan, nan, nan	Best Loss: 2.4374 (4)	Accuracy: 0.00%
22	Validation losses: nan, nan, nan	Best Loss: 2.4374 (5)	Accuracy: 0.00%
23	Validation losses: nan, nan, nan	Best Loss: 2.4374 (6)	Accuracy: 0.00%
24	Validation losses: nan, nan, nan	Best Loss: 2.4374 (7)	Accuracy: 0.00%
25	Validation losses: nan, nan, nan	Best Loss: 2.4374 (8)	Accuracy: 0.00%
26	Validation losses: nan, nan, nan	Best Loss: 2.4374 (9)	Accuracy: 0.00%
27	Validation losses: nan, nan, nan	Best Loss: 2.4374 (10)	Accuracy: 0.00%
28	Validation losses: nan, nan, nan	Best Loss: 2.4374 (11)	Accuracy: 0.00%
29	Validation losses: nan, nan, nan	Best Loss: 2.4374 (12)	Accuracy: 0.00%
30	Validation losses: nan

75	Validation losses: 0.3071, 1.0972, 1.4042	Best Loss: 1.3939 (2)	Accuracy: 90.08%
76	Validation losses: 0.3079, 1.0954, 1.4033	Best Loss: 1.3939 (3)	Accuracy: 89.66%
77	Validation losses: 0.3118, 1.0935, 1.4053	Best Loss: 1.3939 (4)	Accuracy: 89.62%
78	Validation losses: 0.3052, 1.0921, 1.3973	Best Loss: 1.3939 (5)	Accuracy: 90.08%
79	Validation losses: 0.3106, 1.0914, 1.4020	Best Loss: 1.3939 (6)	Accuracy: 90.28%
80	Validation losses: 0.3208, 1.0913, 1.4121	Best Loss: 1.3939 (7)	Accuracy: 90.28%
81	Validation losses: 0.3247, 1.0922, 1.4169	Best Loss: 1.3939 (8)	Accuracy: 90.30%
82	Validation losses: 0.3293, 1.0931, 1.4224	Best Loss: 1.3939 (9)	Accuracy: 90.20%
83	Validation losses: 0.3314, 1.0939, 1.4252	Best Loss: 1.3939 (10)	Accuracy: 90.08%
84	Validation losses: 0.3265, 1.0945, 1.4209	Best Loss: 1.3939 (11)	Accuracy: 90.16%
85	Validation losses: 0.3259, 1.0944, 1.4204	Best Loss: 1.3939 (12)	Accuracy: 89.56%
86	Validation losses: 0.3181, 1.0938, 1.4119	Best Loss: 1.3939 (13)	Accur

75	Validation losses: 0.2965, 1.0982, 1.3946	Best Loss: 1.3946 (0)	Accuracy: 90.10%
76	Validation losses: 0.2999, 1.0961, 1.3960	Best Loss: 1.3946 (1)	Accuracy: 90.06%
77	Validation losses: 0.3001, 1.0942, 1.3944	Best Loss: 1.3944 (0)	Accuracy: 90.18%
78	Validation losses: 0.3069, 1.0930, 1.3999	Best Loss: 1.3944 (1)	Accuracy: 90.02%
79	Validation losses: 0.3111, 1.0922, 1.4033	Best Loss: 1.3944 (2)	Accuracy: 90.68%
80	Validation losses: 0.3131, 1.0921, 1.4052	Best Loss: 1.3944 (3)	Accuracy: 90.34%
81	Validation losses: 0.3158, 1.0929, 1.4087	Best Loss: 1.3944 (4)	Accuracy: 90.16%
82	Validation losses: 0.3111, 1.0940, 1.4051	Best Loss: 1.3944 (5)	Accuracy: 90.56%
83	Validation losses: 0.3143, 1.0946, 1.4089	Best Loss: 1.3944 (6)	Accuracy: 90.40%
84	Validation losses: 0.3153, 1.0954, 1.4107	Best Loss: 1.3944 (7)	Accuracy: 89.96%
85	Validation losses: 0.3397, 1.0946, 1.4343	Best Loss: 1.3944 (8)	Accuracy: 89.34%
86	Validation losses: 0.3152, 1.0944, 1.4096	Best Loss: 1.3944 (9)	Accuracy:

71	Validation losses: 0.2903, 1.1068, 1.3971	Best Loss: 1.3962 (1)	Accuracy: 89.66%
72	Validation losses: 0.2930, 1.1032, 1.3963	Best Loss: 1.3962 (2)	Accuracy: 90.00%
